In [1]:
!pwd

/root/autodl-tmp/generativeModel/disguish_learning/LR_001


In [2]:
import os

In [3]:
os.chdir('/root/autodl-tmp/generativeModel/disguish_learning/LR_001')

In [4]:
import transformers
from transformers import  AutoModelForCausalLM,AutoTokenizer

In [5]:


import torch
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
)
from datasets import load_dataset


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


In [6]:
cache_dir = '/root/autodl-tmp/model/'
model = AutoModelForCausalLM.from_pretrained('bigscience/bloom-3b',cache_dir=cache_dir,trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('bigscience/bloom-3b',cache_dir=cache_dir,trust_remote_code=True)

In [12]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 2560)
        (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-29): 30 x BloomBlock(
            (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): MergedLinear(
                in_features=2560, out_features=7680, bias=True
                (lora_dropout): Dropout(p=0.05, inplace=False)
                (lora_A): Linear(in_features=2560, out_features=16, bias=False)
                (lora_B): Conv1d(16, 5120, kernel_size=(1,), stride=(1,), groups=2, bias=False)
              )
              (dense): Linear(in_features=2560, out_features=2560, bias=True)
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (post_attention_la

In [7]:
# make model as

In [ ]:
model = prepare_model_for_int8_training(model)
lora_config = LoraConfig(
    r=8,#LORA_R,
    lora_alpha=16,#LORA_ALPHA,
    target_modules=["query_key_value"],#TARGET_MODULES,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model,lora_config)

In [13]:
data_cache_dir = '/root/autodl-tmp/data/'

In [14]:
data = load_dataset("cahya/instructions-zh",cache_dir=data_cache_dir)

Using custom data configuration cahya--instructions-zh-507237297bfcf9f5
Found cached dataset parquet (/root/autodl-tmp/data/cahya___parquet/cahya--instructions-zh-507237297bfcf9f5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
for x in data['train']:
    print(x)
    break

{'id': 42216, 'text': 'User: 虚假信息与虚假信息有什么区别?\nAssistant: 误导和误导之间有区别 - 误导是欺骗的使用来破坏沟通过程,而误导是错误的信息或事实的误解。 误导的一个常见例子是新闻来源故意报告虚假声明或误导性信息以传播宣传和影响意见。'}


In [16]:
def split_train_example(text:str):
    answer_prefix = "Assistant:"
    prompt_prefix = "User:"

    answer_start_idx = text.rfind(answer_prefix)
    if(answer_start_idx > 0):
        # this is an trian data
        answer = text[answer_start_idx + len(answer_prefix):]
        prompt = text[:answer_start_idx].replace(prompt_prefix,"")
    else:
        prompt = text
        answer = None

    return prompt,answer

In [17]:
def build_tokenzie_func(tokenizer,pad_idx=0,max_length=256,pad=True):
    def tokenize(example):
        text = example['text']
        prompt,answer = split_train_example(text)
        prompt_idxs = tokenizer(prompt)
        answer_idxs = tokenizer(answer) if answer is not None else []

        example = {}
        example['input_ids'] = prompt_idxs['input_ids'] + answer_idxs['input_ids']
        example['attention_mask'] = prompt_idxs['attention_mask'] + answer_idxs['attention_mask']
        example['labels'] = [0] * len(prompt_idxs['attention_mask']) + answer_idxs['input_ids']
        example['prompt'] = prompt
        example['answer'] = answer
        
        if(len(example['input_ids']) < max_length):
            count = max_length - len(example['input_ids'])
            example['input_ids'] = example['input_ids'] + [0] * count
            example['attention_mask'] = example['attention_mask']+ [0] * count
            example['labels'] = example['labels']+ [0] * count
        
        example['input_ids'] = example['input_ids'][:max_length]
        example['attention_mask'] = example['attention_mask'][:max_length]
        example['labels'] = example['labels'][:max_length]
        

        return example
    return tokenize
tokenize_func = build_tokenzie_func(tokenizer)

In [18]:
train_data = data['train'].map(tokenize_func)

  0%|          | 0/76886 [00:00<?, ?ex/s]

In [19]:
print(train_data[0])

{'id': 42216, 'text': 'User: 虚假信息与虚假信息有什么区别?\nAssistant: 误导和误导之间有区别 - 误导是欺骗的使用来破坏沟通过程,而误导是错误的信息或事实的误解。 误导的一个常见例子是新闻来源故意报告虚假声明或误导性信息以传播宣传和影响意见。', 'input_ids': [210, 238785, 8401, 1961, 238785, 8401, 37242, 49076, 1021, 210, 19651, 5092, 745, 19651, 5092, 10040, 706, 49076, 647, 210, 19651, 5092, 644, 151711, 60697, 1572, 30949, 28543, 5425, 2818, 15, 1518, 19651, 5092, 644, 35892, 42287, 2140, 28866, 373, 201933, 420, 210, 19651, 5092, 16368, 39428, 40567, 644, 19312, 28349, 70251, 4840, 238785, 32460, 2140, 19651, 5092, 1963, 8401, 909, 35786, 26262, 745, 7962, 15344, 420, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [20]:
val_data = data['validation'].map(tokenize_func)
test_data = data['test'].map(tokenize_func)

  0%|          | 0/2023 [00:00<?, ?ex/s]

  0%|          | 0/2024 [00:00<?, ?ex/s]

In [21]:
ddp = True if torch.cuda.device_count() > 1 else False
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        num_train_epochs=1,
        learning_rate=1e-5,
        fp16=True,
        logging_steps=2000,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=2000,
        save_steps=2000,
        output_dir="lora-alpaca",
        save_total_limit=3,
        load_best_model_at_end=True,
        ddp_find_unused_parameters=False if ddp else None,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())
).__get__(model, type(model))


In [22]:
if torch.__version__ >= "2":
    model = torch.compile(model)

##############
# 训练循环

trainer.train()


model.save_pretrained("lora-alpaca")

print("\n If there's a warning about missing keys above, please disregard :)")

/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 23.69 GiB total capacity; 13.18 GiB already allocated; 362.94 MiB free; 13.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# 测试部分

In [9]:
prompt = "您好"

In [18]:
prompt_idxs = tokenizer(prompt)

prompt_idxs = {
    key : torch.LongTensor([prompt_idxs[key]])#.cuda()
    for key in prompt_idxs
}


In [19]:
prompt_idxs['input_ids']

tensor([[127136]])

In [20]:
prompt_idxs['input_ids'].max()

tensor(127136)

In [21]:
prompt_idxs['input_ids'].min()

tensor(127136)

In [22]:
model.disable_adapter()

In [23]:
for n,x in model.base_model.model.named_parameters():
    if("embedding" in n):
        print(x.size())

torch.Size([250880, 2560])
torch.Size([2560])
torch.Size([2560])


In [24]:

with torch.no_grad():
   x = model.generate(input_ids=prompt_idxs['input_ids'],max_new_tokens=128)


In [25]:
for z in x.cpu().numpy():   
    y= tokenizer.decode(z)
    print(y)

您好，您现在可以随时从“一带一路”倡议的框架内保持与国际产能和装备等领域的合作，也可以扩大与沿线国家的合作范围。， 报道称，马斯说：美国退出伊核协议威胁到一整套国际性的冲突解决措施，可为我们的远邻(中东地区)带来不可预见的后果。。 报道称，韩国外交部韩美防卫费分担谈判代表张元三和美方谈判代表蒂莫西·贝茨等在为期两天的会议上就签署《防卫费分担特别协定》(SMA)，规定2019年以后驻韩美军驻扎费用的分担比例问题进行了第四轮讨论。。
中国
